### This file contains implementations of the methods necessary to implement a genetic algorithm to search for an optimal racetrack route
1) Initialization method: Creates a new RacetrackCommandNodeSeries object

In [ ]:
import random

In [ ]:
#How many waypoint indices between command points. 
#For long tracks with few curves, this should be calibrated to a higher number
#For tracks that are twisty or with few nodes, it should be a lower number
init_cp_frequency = 20
verbosity = 1

In [ ]:
#Define and assign our genetic algorithm methods
#Dependent on initialization of a global racetrack object of type Racetrack
def initializationMethod():
    command_node_series = RacetrackCommandNodeSeries(racetrack, init_cp_frequency=init_cp_frequency, verbosity=verbosity)
    command_node_series.generateRandomCommandNodes()
    return command_node_series

In [ ]:
#"Cut out" the same section of the route from each crossover racetrack and swap them between command node series objects
def crossoverMethod(racetrackCommandNodeSeries1, racetrackCommandNodeSeries2):
    inner_waypoints_len = len(racetrackCommandNodeSeries1.racetrack.waypoints_inner)
    wp_1_idx = random.randint(0,inner_waypoints_len)
    wp_2_idx = random.randint(0,inner_waypoints_len)
    #Re-roll if both cutpoints are same index
    while wp_1_idx == wp_2_idx:
        wp_2_idx = random.randint(0,inner_waypoints_len)
        
    #Sort so that wp_1 < wp_2   
    if wp_2_idx < wp_1_idx:
        wp_3_idx = wp_2_idx
        wp_2_idx = wp_1_idx
        wp_1_idx = wp_3_idx
        
    #Get start and end indices of sub-arrays within wp_1_idx and wp_2_idx. Calculate command node cutpoints based on waypoint cutpoints
    cns1_start_idx = -1
    cns1_end_idx = -1
    for idx, command_node in enumerate(racetrackCommandNodeSeries1.command_nodes):
        if command_node[0] >= wp_1_idx and cns1_start_idx == -1: cns1_start_idx = idx
        if command_node[0] >= wp_2_idx and cns1_end_idx == -1: cns1_end_idx = idx
    cns2_start_idx = -1
    cns2_end_idx = -1
    for idx, command_node in enumerate(racetrackCommandNodeSeries2.command_nodes):
        if command_node[0] >= wp_1_idx and cns2_start_idx == -1: cns2_start_idx = idx
        if command_node[0] >= wp_2_idx and cns2_end_idx == -1: cns2_end_idx = idx     
    #Get references to sub-arrays within wp_1_idx and wp_2_idx then swap them
    cns_sublist_1 = racetrackCommandNodeSeries1.command_nodes[cns1_start_idx:cns1_end_idx]
    cns_sublist_2 = racetrackCommandNodeSeries2.command_nodes[cns2_start_idx:cns2_end_idx]
    racetrackCommandNodeSeries1.command_nodes[cns1_start_idx:cns1_end_idx] = cns_sublist_2
    racetrackCommandNodeSeries2.command_nodes[cns2_start_idx:cns2_end_idx] = cns_sublist_1
    racetrackCommandNodeSeries1.validateAndFixObstructions()
    racetrackCommandNodeSeries2.validateAndFixObstructions()
    
    return racetrackCommandNodeSeries1, racetrackCommandNodeSeries2

In [ ]:
#Define fitness method as route traversal time approximation
def fitnessMethod(a):
    return a.routeFitness()

In [ ]:
# Stochastic selection from among 3 mutation operations
# 0=Delete a command node
# 1=Add a new command node
# 2=Modify distance of an existing command node
def mutationMethod(racetrackCommandNodeSeries): 
    mutation_op = random.randint(0,2)
    mutation_node_idx = random.randint(1,len(racetrackCommandNodeSeries.command_nodes)-2)
    if mutation_op == 0:
        del racetrackCommandNodeSeries.command_nodes[mutation_node_idx]
    if mutation_op == 1:
        mutation_node_idx = random.randint(0,len(racetrackCommandNodeSeries.racetrack.waypoints_inner)-1)
        racetrackCommandNodeSeries.generateRandomCommandNodeAtIndex(mutation_node_idx)
    if mutation_op == 2:
        track_width = racetrackCommandNodeSeries.racetrack.trackwidth
        dist_from_inner_waypoint = np.random.uniform(0, track_width/2) + np.random.uniform(0, track_width/2)
        racetrackCommandNodeSeries.command_nodes[mutation_node_idx][1] = dist_from_inner_waypoint
    racetrackCommandNodeSeries.validateAndFixObstructions()

In [ ]:
#Print the best score and plot the best route for each generation
def reportingMethod(phenotype_score_arr):
    #print('Best route score:', phenotype_score_arr[0][1])
    phenotype_score_arr[0][0].plotTrackAndRoute()
    print('Top 5 route fitness scores')
    print('--------------------------')
    for idx, phenotype_score_tuple in enumerate(phenotype_score_arr):
        if idx > 5: break; #5 is enough
        print((idx+1),":",phenotype_score_tuple[1])